In [1]:
import requests      # Библиотека работы с HTTP-запросами по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами
import re            # Для работы с регулярными выражениями

In [2]:
def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description

# Чтение файла vacancies.json и создание словаря vacancies_dict
with open('2/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)
vacancies_dict = {vacancy["id"]: "" for vacancy in vacancies}

# Обход словаря vacancies_dict и заполнение значениями ключа "description"
for vacancy_id in vacancies_dict:
    vacancies_dict[vacancy_id] = get_description(vacancy_id)

# Сохранение результата в файл result.json
with open('2/result.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies_dict, f, ensure_ascii=False, indent=4)

Ошибка получения описания вакансии 78534080. Повтор запроса через 5 секунд.


In [4]:
vacancies_dict

{'79110745': '',
 '79120615': '',
 '78934984': '',
 '78954091': '',
 '79190614': '',
 '79084256': '',
 '78613909': '',
 '79315237': '',
 '78968207': '',
 '79172870': '',
 '79192759': '',
 '79226963': '',
 '78370985': '',
 '78340750': '',
 '79148004': '',
 '79162833': '',
 '78879859': '',
 '79323507': '',
 '79261592': '',
 '78448054': '',
 '78952166': '',
 '78687348': '',
 '79115441': '',
 '78427104': '',
 '79145131': '',
 '77577033': '',
 '79122630': '',
 '79147234': '',
 '78852846': '',
 '78519105': '',
 '78273747': '',
 '78029161': '',
 '79089912': '',
 '73888449': '',
 '79226964': '',
 '79314682': '',
 '79224520': '',
 '79090742': '',
 '79279390': '',
 '79174677': '',
 '79305476': '',
 '78664757': '',
 '79127417': '',
 '79232504': '',
 '77426640': '',
 '79221567': '',
 '78879858': '',
 '78258371': '',
 '79314135': '',
 '78306430': '',
 '71220088': '',
 '79174702': '',
 '75825082': '',
 '78617302': '',
 '79179730': '',
 '79267194': '',
 '78386771': '',
 '79006335': '',
 '67854831': '

In [ ]:
# Добавление ключа "description" для каждой вакансии
for vacancy in vacancies:
    vacancy["description"] = ""

# Получение списка id вакансий из ответа на запрос
vacancy_ids = [vacancy["id"] for vacancy in vacancies]

# Цикл по каждому id вакансии
for vacancy_id in vacancy_ids:
    

    # Запрос описания вакансии
    response = requests.get('https://api.hh.ru/vacancies/' + vacancy_id)
    
    # Получение описания вакансии из ответа на запрос
    description = json.loads(response.text).get("description", "")

    
    # Запись описания вакансии в ключ "description"
    for vacancy in vacancies:
        if vacancy["id"] == vacancy_id:
            vacancy["description"] = description

In [ ]:
# Получаем описания для каждой вакансии датафрейма, записываем результат

def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description

# Применяем функцию для каждой вакансии в датафрейме и записываем результат в новый столбец
df['description'] = df['id'].apply(get_description)

In [ ]:
URL = 'https://api.hh.ru/vacancies'

params = {
    'text': "Data Scientist",
    'area': 1,
    'page': 0,
    'per_page': 10
}

req = requests.get(URL, params)
data = json.loads(req.content.decode())

In [ ]:
data.keys()

In [ ]:
# Посмотрим описание первой вакансии
data

In [ ]:
# Сколько найдено вакансий
data['found']

In [ ]:
# Страниц в результатах поиска
data['pages']

In [ ]:
# Сделаем так, чтобы выводились все столбцы датафрейма
pd.set_option('display.max_columns', None)

С помощью метода pandas.json_normalize разберем структурированные данные из JSON в табличный формат.

In [ ]:
df = pd.json_normalize(data['items'])
df.head()

In [ ]:
df.shape

Видим, что в столбце 'professional_roles' данные не нормализовались. Что бы разобрать вложенный список из professional_roles, применим к столбцу лямбда-функцию, разделим его на два новых столбца:

In [ ]:
df[['professional_roles_id', 'professional_roles_name']] = (
    df['professional_roles']
    .apply(lambda x: pd.Series([x[0]['id'], x[0]['name']]))
)

In [ ]:
df.head()

In [ ]:
# Выведем названия столбцов
print(df.columns)

In [ ]:
# Предвинем два получившихся столбца на место изначального professional_roles, 
# а его не будем включать в обновленный датафрейм 

df=df[['id', 'premium', 'name', 'has_test', 'response_letter_required',
       'address', 'response_url', 'sort_point_distance', 'published_at',
       'created_at', 'archived', 'apply_alternate_url', 'insider_interview',
       'url', 'adv_response_url', 'alternate_url', 'relations', 'contacts',
       'schedule', 'working_days', 'working_time_intervals',
       'working_time_modes', 'accept_temporary', 'professional_roles_id',
       'professional_roles_name', 'accept_incomplete_resumes',
       'department.id', 'department.name', 'area.id', 'area.name', 'area.url',
       'salary.from', 'salary.to', 'salary.currency', 'salary.gross',
       'type.id', 'type.name', 'employer.id', 'employer.name', 'employer.url',
       'employer.alternate_url', 'employer.logo_urls.240',
       'employer.logo_urls.90', 'employer.logo_urls.original',
       'employer.vacancies_url', 'employer.trusted', 'snippet.requirement',
       'snippet.responsibility', 'department', 'employer.logo_urls',
       'address.city', 'address.street', 'address.building', 'address.lat',
       'address.lng', 'address.description', 'address.raw', 'address.metro',
       'address.metro_stations', 'address.id', 'salary',
       'address.metro.station_name', 'address.metro.line_name',
       'address.metro.station_id', 'address.metro.line_id',
       'address.metro.lat', 'address.metro.lng']]

In [ ]:
df.head()

In [ ]:
df['snippet.requirement'].iloc[1]

Также видим, что в столбцах snippet.requirement	и snippet.responsibility есть теги. Если в тексте снипета встретилась поисковая фраза (параметр text ), она будет подсвечена тегом highlighttext (из документации по API). Но нам эти теги ни к чему, избавимся от них:

In [ ]:
def remove_tags(text):
    if isinstance(text, str):
        return re.sub(r'<.*?>', '', text)
    else:
        return text

df[['snippet.requirement', 'snippet.responsibility']] = df[['snippet.requirement', 'snippet.responsibility']].applymap(remove_tags)


In [ ]:
df['snippet.requirement'].iloc[1]

In [ ]:
df['snippet.responsibility'].iloc[1]

Для получения полного описания вакансии потребуется задать отдельный запрос, используя ее id.

In [ ]:
vacancy = df['id'].iloc[0]
vacancy_url = f'https://api.hh.ru/vacancies/{vacancy}'

req = requests.get(vacancy_url)
vacancy_info = json.loads(req.content.decode())
vacancy_info